### 2.1 신경망과의 첫 만남

MNIST 데이터셋
--> 이곳에 MNIST 데이터셋에 대해 책 또는 인터넷 참고/조사한 내용을 기술


# MNIST dataset

    MNIST : Mixed National Institute of Standards and Technology database

* train-images-idx3-ubyte.gz	학습 셋 이미지 - 55000개의 트레이닝 이미지, 5000개의 검증 이미지
* train-labels-idx1-ubyte.gz	이미지와 매칭되는 학습 셋 레이블
* t10k-images-idx3-ubyte.gz	테스트 셋 이미지 - 10000개의 이미지
* t10k-labels-idx1-ubyte.gz	이미지와 매칭되는 테스트 셋 레이블

컴퓨터에게는 0이 그려진 이미지는 단지 픽셀 밝기 값(Intensity)으로 구성된 2차원 행렬일 뿐이다.


# 데이터 전처리 작업

input_data.py	학습과 추정을 위한 MNIST 데이터셋을 다운로드하는 코드

파일들 자체는 표준 이미지 포맷이 아니며 직접 input_data.py에 있는 extract_images()와 extract_labels() 함수를 사용하여 언패킹할 수 있습니다.

이미지 데이터는 [image index, pixel index] 형태의 이차원 텐서(여기선 2차원 배열을 의미함)로 추출될 수 있습니다. 각 엔트리는 특정 이미지에서 특정 픽셀의 휘도값이며, [0, 255]에서 [0, 1]까지 재조정됩니다. "image index"는 데이터셋에 있는 이미지를 가리키며, 0부터 데이터셋의 크기까지 카운팅됩니다. 그리고 "pixel index"는 어떤 이미지에서의 특정 픽셀을 가리키며, 0부터 이미지에 존재하는 픽셀의 갯수까지 존재합니다.

train-*파일들에 있는 60000개의 예시들은 학습을 위한 55000개의 예시들과 검증을 위한 5000개의 예시들로 나뉘어집니다. 데이터셋에 있는 모든 28x28 픽셀의 그레이스케일 이미지의 크기는 784이고 따라서 학습 셋 이미지를 위한 출력값 텐서는 [55000, 784]의 형태가 됩니다.

이 기본 코드는 다운로드와 압축풀기 그리고 다음의 데이터셋들을 위해 이미지와 레이블을 변형할 것입니다.

            데이터셋	목적
    data_sets.train	초기 학습을 위한 55000개의 이미지들과 레이블들
    data_sets.validation 학습 정확도의 반복적 검증을 위한 5000개의 이미지와 레이블들
    data_sets.test	학습 정확도의 마지막 테스팅을 위한 10000개의 이미지와 레이블들

read_data_sets()함수는 각 세가지 데이터 셋을 위한 DataSet인스턴스를 가진 딕셔너리를 리턴합니다. 
DataSet.next_batch()메서드는 batch_size개의 이미지 리스트와 레이블들로 이루어진 튜플을 실행중인 TensorFlow 세션에 넣기위해 사용될 수 있습니다.


# 인터넷 참고

1. https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/tutorials/mnist/download/

2. http://solarisailab.com/archives/303



#### 코드 2-1 케라스에서 MNIST 데이터셋 적재하기

In [2]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

Using TensorFlow backend.


#### 코드 2-2 신경망 구조

In [3]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28*28,)))
network.add(layers.Dense(10, activation='softmax'))

W0722 12:09:48.841917  1496 deprecation_wrapper.py:119] From C:\Users\ehddn\.conda\envs\venv\lib\site-packages\keras-2.2.4-py3.7.egg\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0722 12:09:48.861864  1496 deprecation_wrapper.py:119] From C:\Users\ehddn\.conda\envs\venv\lib\site-packages\keras-2.2.4-py3.7.egg\keras\backend\tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0722 12:09:48.864870  1496 deprecation_wrapper.py:119] From C:\Users\ehddn\.conda\envs\venv\lib\site-packages\keras-2.2.4-py3.7.egg\keras\backend\tensorflow_backend.py:4420: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



#### 코드 2-3 컴파일 단계

In [4]:
network.compile(optimizer='rmsprop',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

W0722 12:09:49.314308  1496 deprecation_wrapper.py:119] From C:\Users\ehddn\.conda\envs\venv\lib\site-packages\keras-2.2.4-py3.7.egg\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0722 12:09:49.334256  1496 deprecation_wrapper.py:119] From C:\Users\ehddn\.conda\envs\venv\lib\site-packages\keras-2.2.4-py3.7.egg\keras\backend\tensorflow_backend.py:3564: The name tf.log is deprecated. Please use tf.math.log instead.



#### 코드 2-4 이미지 데이터 준비하기

In [5]:
train_images = train_images.reshape ((60000, 28*28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype('float32') / 255

#### 코드 2-5 레이블 준비하기

In [6]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

#### 훈련

In [7]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

W0722 12:09:51.372319  1496 deprecation.py:323] From C:\Users\ehddn\.conda\envs\venv\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0722 12:09:51.417808  1496 deprecation_wrapper.py:119] From C:\Users\ehddn\.conda\envs\venv\lib\site-packages\keras-2.2.4-py3.7.egg\keras\backend\tensorflow_backend.py:1021: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/5
60000/60000 [==============================] - 3s 51us/step - loss: 0.2560 - acc: 0.9262
Epoch 2/5
60000/60000 [==============================] - 3s 49us/step - loss: 0.1045 - acc: 0.9693
Epoch 3/5
60000/60000 [==============================] - 3s 49us/step - loss: 0.0683 - acc: 0.9793
Epoch 4/5
60000/60000 [==============================] - 3s 49us/step - loss: 0.0498 - acc: 0.9847
Epoch 5/5
60000/60000 [==============================] - 3s 49us/step - loss: 0.0378 - acc: 0.9887


#### 테스트

In [8]:
test_loss, test_acc = network.evaluate(test_images, test_labels)
print('test_acc:', test_acc)

10000/10000 [==============================] - 0s 31us/step
test_acc: 0.9797
